# Análisis de sueldos con Python

## Data

* [2021.2 - sysarmy - Encuesta de remuneración salarial Argentina
](https://docs.google.com/spreadsheets/d/1-ZRznwS4TK74o90aOiCKS5SiXxUQ2buN1wxZIMHQmzQ/edit#gid=557755710) 
* [2021.2 - sysarmy - Encuesta de remuneración salarial Latam
](https://docs.google.com/spreadsheets/d/1BkBNt1MHVS7DeIlpgmK9l6krtSQ5t_olRhlcyxMeKy0/edit#gid=557755710)

1. Descargar, leer y unir en una sola estructura de datos. ¿Qué estructura elegiste y por qué?

In [ ]:
# En este primer punto se traerán los datos de los dos archivos para trabajar.
# Se ordenarán de tal forma que simplifique el trabajo posterior, así como la normalizacion de sus valores, creando funciones para simplificar el trabajo.


In [ ]:
# A continuación la importación de la encuesta de Latam y la definición de su estructura que será listas de diccionarios.

In [292]:
import csv

list_keys = []
list_values = []
list_dict_info_latam = []                                         #Acá se guarda la info de los dict de Latam

with open('2021.2 - sysarmy - Encuesta de remuneración salarial Latam - Latam.tsv', encoding="utf8") as s_latam:
    latam_reader = csv.reader(s_latam, delimiter='\t')
    for index, line in enumerate(latam_reader):
        #if index > 9:
            #print(f'{latam_line}')
        # Líneas 0 a 9 vacías
        # Líneas 10  -> preguntas
        # Líneas 11+ -> respuestas
        if index == 10:
            list_keys = line
            list_keys[0] = 'País'                                 #La pregunta Estoy trabajando en se pasa a llamar País
            
        elif index >= 11:
            dict_info = {}
            list_values = line
            for i, key in enumerate(list_keys):
                dict_info[list_keys[i]] = list_values[i]
            
            dict_info['Ciudad'] = dict_info[dict_info['País']]
            
            list_colums_to_remove = ['Bolivia', 'Chile', 'Colombia', 'Costa Rica', 'Cuba', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', 'México', 'Nicaragua', 'Panamá', 'Paraguay', 'Perú', 'Puerto Rico', 'República Dominicana', 'Uruguay', 'Venezuela']
            for i, key in enumerate(list_colums_to_remove):       #Para borrar las columnas de los países con sus regiones
                dict_info.pop(key)
            
            list_dict_info_latam.append(dict_info)
            


In [ ]:
# A continuación la importación de la encuesta de Argentina y su definición de estructura que será listas de diccionarios.

In [293]:
import csv

list_keys = []
list_values = []
list_dict_info_arg = []                                          #Acá se guarda la info de los dict de Argentina

with open('2021.2 - sysarmy - Encuesta de remuneración salarial Argentina - Argentina.tsv', encoding="utf8") as s_arg:
    arg_reader = csv.reader(s_arg, delimiter='\t')
    for index, line in enumerate(arg_reader):
        #if index > 8:
            #print(f'{arg_line}')
        # Líneas 0 a 8 vacías
        # Líneas 9   -> preguntas
        # Líneas 10+ -> respuestas
        if index == 9:
            list_keys = line
            list_keys[0] = 'Ciudad'                             #La pregunta Dónde estás trabajando se pasa a llamar Ciudad

        elif index >= 10:
            dict_info = {}
            list_values = line
            for i, key in enumerate(list_keys):
                dict_info[list_keys[i]] = list_values[i]
            dict_info['País'] = 'Argentina'                     #Se agrega una pregunta País con respuesta Argentina
            
            list_dict_info_arg.append(dict_info)
                            

In [ ]:
# A continuación seran unidas las dos estructuras en una sola. Argentina se agregará a Latam.

In [296]:
list_all = list_dict_info_latam + list_dict_info_arg

print('Para verificar unión tomando el len de cada lista:')
print(f'{len(list_dict_info_latam)} + {len(list_dict_info_arg)} = {len(list_all)}')

Para verificar unión tomando el len de cada lista:
429 + 6440 = 6869


In [ ]:
# Importación de LIBRERÍAS a utilizar:

In [297]:
import statistics
from statistics import median

In [ ]:
# Definiciones de FUNCIONES a utilizar:

In [372]:
# Función que devuelve el porcentaje.

def percentage(total,quantity):
    return round(quantity * 100  / total, 2)





# Función que calcula la paticipación de una columna ordenada descendiente.

def participation_per_key_DESC(list_to_analyze, key):
    dict_result_Participacion = {}

    for record in list_to_analyze:
        if record[key] != '':
            if record[key] in dict_result_Participacion:
                #Si la ciudad ya esta en el diccionario resultado, incremento el contador
                dict_result_Participacion[record[key]] += 1 
            else:
                #Si la ciudad NO esta en el diccionario resultado, inicializo el contador en 1
                dict_result_Participacion[record[key]] = 1   

    return sorted(dict_result_Participacion.items(), key=lambda x: x[1], reverse=True)





# Función que calcula la particpación de dos claves, ordenada descendiente. 

def participation_per_keys_Order_by_Keys(list_to_analyze, key1, key2):
    dict_result_Participacion = {}

    for record in list_to_analyze:
        if record[key1] != '':
            if (record[key1], record[key2]) in dict_result_Participacion:
                #Si la clave ya esta en el diccionario resultado, incremento el contador
                dict_result_Participacion[(record[key1], record[key2])] += 1 
            else:
                #Si la clave NO esta en el diccionario resultado, inicializo el contador en 1
                dict_result_Participacion[(record[key1], record[key2])] = 1   

    return sorted(dict_result_Participacion.items(), key=lambda x: x[0], reverse=False)





# Función para calcular la suma de las cantidades de una lista de tuplas (clave, cantidad).

def participation_count(tuple_list):
    total_count = 0
    for tuple_key_participation in tuple_list:
        total_count += tuple_key_participation[1]
    return total_count





# Función normalización Si - No - No contesta.

def normalize_yes_no_empty(value):
    if value == '':
        normalized_value = 'No contesta'
    else:
        normalized_value = value
        
    return normalize_STR(normalized_value)

#probar función normalize_yes_no_empty:
#normalize_yes_no_empty('')






# Función para normalizar STR.

def normalize_STR(value):
    normalized_value = str(value).strip().capitalize()
    normalized_value = normalized_value.replace('á', 'a')
    normalized_value = normalized_value.replace('é', 'e')
    normalized_value = normalized_value.replace('í', 'i')
    normalized_value = normalized_value.replace('ó', 'o')
    normalized_value = normalized_value.replace('ú', 'u')
    normalized_value = normalized_value.replace('ñ', 'n')
    return normalized_value

#probar función normalize_STR:
#normalize_STR('agAáéíóúSgiaASs')





# Función para normalizar valores a Float.

def normalize_FLOAT(value):
    normalized_value = round(float(value),2)
    return normalized_value

#probar función normalize_FLOAT:
#normalize_FLOAT('1.1123123')





# Función para normalizar valores a INT (de STR a INT). 

def normalize_INT(value):
    normalized_value = int(normalize_FLOAT(value))
    return normalized_value

#probar función normalize_INT:
#normalize_INT('19.0')





# Función para normalizar la columna Carrera.

def normalize_carrera(value):
    normalized_value = str(value).lower()
    if normalized_value in ['.', '-', 'x', 'no', 'cp', 'n/a', 'no estudie', 'ninguna', 'ninguna.', 'ninguna de las anteriores']:
        normalized_value = 'Otros'
    normalized_value = normalized_value.replace('lic.', 'Licenciatura ')
    normalized_value = normalized_value.replace('lic ', 'Licenciatura ')
    normalized_value = normalized_value.replace('tec.', 'Tecnicatura ')
    normalized_value = normalized_value.replace('tec ', 'Tecnicatura ')
    normalized_value = normalized_value.replace('cs.', 'Ciencias ')
    normalized_value = normalized_value.replace('cs ', 'Ciencias ')
    normalized_value = normalized_value.replace('ed.', 'Educacion ')
    normalized_value = normalized_value.replace('ed ' , 'Educacion ')
    normalized_value = normalized_value.replace('rrii', 'Relaciones Internacionales ')
    normalized_value = normalized_value.replace('rrhh', 'Recursos humanos ')
    normalized_value = normalized_value.replace('ing.', 'Ingeniero ')
    normalized_value = normalized_value.replace('ing ', 'Ingeniero ')
    normalized_value = normalized_value.replace('mg.', 'Magister ')
    normalized_value = normalized_value.replace('arq.', 'Arquitecto ')
    normalized_value = normalized_value.replace('arq ', 'Arquitecto ')
    normalized_value = normalize_STR(normalized_value)
    return normalized_value

#probar función normalize_carrera:
#normalize_carrera('lic. VhnkYghs')
#normalize_carrera('Arq.')





# Función para normalizar la columna Discapacidad.

def normalize_discapacidad(value):
    normalized_value = str(value).lower().strip()
    if normalized_value in ['', 'no,','ninguna,', 'ninguna', 'no', 'ninguno', 'no aplica', 'visceral', 'soy', 'none', 'pija larga', 'síndrome del pene grande', 'no me gusta el fútbol xd', '0','de no entender como en la respuesta anterior no pusieron una respuesta libre. soy furro por eleccion y conviccion']:
        normalized_value = 'No'
    #else:
        #print(normalized_value)
    #normalized_value = normalized_value.replace('lic.', 'Licenciatura ')
       
    if 'mental' in normalized_value:
        normalized_value = 'Mental'
    if 'visual' in normalized_value or 'lentes' in normalized_value or 'daltonismo' in normalized_value or 'daltonico' in normalized_value or 'anteojos' in normalized_value:
        normalized_value = 'Visual'
    if 'auditiva' in normalized_value or 'audicion' in normalized_value:
        normalized_value = 'Auditiva'
    if 'peso' in normalized_value or 'obesidad' in normalized_value:
        normalized_value = 'Obesidad'   
    if 'autista' in normalized_value or 'autusmo' in normalized_value or 'autismo' in normalized_value:
        normalized_value = 'Autismo'    
 
    normalized_value = normalize_STR(normalized_value)
    return normalized_value

#probar función normalize_discapacidad:
#normalize_discapacidad('')






# Función para calcular medianas.

def get_mediana(list_to_process):
    normalized_value = normalize_FLOAT(statistics.median(list_to_process))
    return normalized_value

#probar función get_mediana:
#test_median = [1, 43, 765, 7896]
#get_mediana(test_median)





# Función para dolarizar valores.

dict_dollars = {'Argentina': 115.18, 'Bolivia': 6.87, 'Chile': 856.26, 'Colombia': 3970.97, 'Costa Rica': 661.18, 'Cuba': 23.98, 'Ecuador': 25000.00, 'El Salvador': 9.74, 'Guatemala': 7.65,'Honduras': 24.53, 'México': 20.55,'Nicaragua': 35.79, 'Panamá': 1.00, 'Paraguay': 6834.03, 'Perú': 3.83, 'Puerto Rico': 1.00 ,'República Dominicana': 55.10, 'Uruguay': 40.89, 'Venezuela': 248303.10}
#Tipos de cambios al 28 abril 2022

def dollarize(amount, country):
    dolarized_amount = normalize_FLOAT(amount) / dict_dollars[country]
    return dolarized_amount

#probar función dollarize:
#dollarize(3000, 'Bolivia')
#dict_dollars['Bolivia']
#type(dict_dollars['Bolivia'])





# Función para normalizar montos en moneda local.

def normalize_amount(amount,country):
    new_amount = ''.join([i for i in amount if i.isdigit() or i in [',','.']])
    if new_amount in ['','.',',']:
        new_amount = ''
    else:
        new_amount = normalize_number_with_puntuaction(new_amount) 
        new_amount = dollarize(new_amount, country)
        new_amount = round(new_amount,2)
    return new_amount

#probar función normalize_amount:
#normalize_amount('Cobro 100.1$','Argentina')
#normalize_amount('Cobro 2.173.689','Puerto Rico')




#Función para normalizar puntuación en valores numéricos.

def normalize_number_with_puntuaction(number):
    if ',' in number and '.' in number:
        if number.find(',') > number.find('.'):
            number = number.replace('.','')
            number = number.replace(',','.')
        else:
            number = number.replace(',','')    
    else:
        number = number.replace(',','.')
   
        if len(number.split('.')) > 2:
            number = number.replace('.','')
    return number
   
#probar función normalize_number_with_puntuaction:
#normalize_number_with_puntuaction('875.673.345')





# Función para obtener una lista de valores asociados al valor de una columna.

def dict_values_por_column(list_to_analyze, column_key, value_key):
    dict_result_values_por_column = {}

    for record in list_to_analyze:
        if record[column_key] != '' and record[value_key] != '':  #Si el campo o el valor son vacíos, no los agrego al resultado
            if record[column_key] in dict_result_values_por_column:
                #Si el valor para column_key ya esta en el diccionario resultado, se agrega a la lista
                dict_result_values_por_column[record[column_key]].append(record[value_key]) 
            else:
                #Si el valor para column_key NO esta en el diccionario resultado, se crea la lista
                dict_result_values_por_column[record[column_key]] = [record[value_key]]

    return dict_result_values_por_column    

#probar función dict_values_por_column:
#dict_values_por_column(list_all_normalized, 'País', 'Ciudad')






# Función de normalización de la columna Me Identifico.

def normalize_gender(value):
    gender = normalize_STR(value).lower()
    if 'trans' in gender or 'travesti' in gender or 'transgenero' in gender:
        gender = 'Transgénero'
    elif 'varon' in gender or 'hombre' in gender or 'masculino' in gender or 'macho' in gender:
        gender = 'Hombre'
    elif 'mujer' in gender: 
        gender = 'Mujer'
    elif 'no binario' in gender or 'no binarie' in gender: 
        gender = 'No binario'
    elif 'prefiero no decir' in gender: 
        gender = 'Prefiere no decir'
    else:
        gender = 'Inválido'
    return gender

#probar función normalize_gender:
#normalize_gender('trans')





# Función para transformar un porcentaje en una barra para usar en gráficos.

def number_to_barchart(number):
    int_number = int(number)
    str_bar = ""
    for i in range(int_number):
        str_bar += '-'
    return str_bar

#probar función number_to_barchart:        
#print(number_to_barchart(10))
#print(number_to_barchart(10.1))
#print(number_to_barchart(20))





#Función para agregar espacios después de un STR hasta que quede de un largo fijo.

def add_spaces(value, until_position):
    result = value
    for i in range(until_position):
        if i >= len(value):
            result += ' '
    #result += '|'
    return result

#probar función add_spaces: 
#print(add_spaces('texto',10))
#print(add_spaces('uno',10))
#print(add_spaces('Virginia',10))



In [ ]:
# A continuación se NORMALIZARÁN los elementos, a través de un función:

In [373]:
# Función de normalización general.

def normalize_record(dict_record):
    dict_result = {}
    
    for i, key in enumerate(dict_record.keys()):
        if key == 'Ciudad':
            if dict_record[key] == '':
                dict_result[key] = 'SIN CIUDAD'
            else:
                dict_result[key] = normalize_STR(dict_record[key])
        elif key == '¿Contribuís a proyectos open source?' or key == '¿Programás como hobbie?':
            dict_result[key] = normalize_yes_no_empty(dict_record[key])
        elif key == 'Años de experiencia':
            #print(dict_record[key])
            dict_result[key] = normalize_FLOAT(dict_record[key])
        elif key in ['Años en la empresa actual', 'Años en el puesto actual']:
            dict_result[key] = normalize_INT(dict_record[key])
        elif key == 'Carrera':
            dict_result[key] = normalize_carrera(dict_record[key])
        elif key == 'Realizaste cursos de especialización' or key == 'Trabajo de' or key == 'Nivel de estudios alcanzado': 
            dict_result[key] = normalize_STR(dict_record[key])
        elif key == 'Me identifico':
            dict_result[key] = normalize_gender(dict_record[key])
        elif key == '¿Tenés algún tipo de discapacidad?':
            dict_result[key] = normalize_discapacidad(dict_record[key])     
        elif key == 'Salario mensual o retiro BRUTO (en tu moneda local)':
            dict_result[key] = normalize_amount(dict_record[key], dict_record['País'])
        else:
            dict_result[key] = dict_record[key]
      
    return dict_result

list_all_normalized = []
for i, key in enumerate(list_all):
    list_all_normalized.append(normalize_record(list_all[i]))
    
#print(list_all_normalized[0].keys())

register_total = len(list_all_normalized)

print('Para verificar que se cambiaron bien los registros:')    
print(list_all[0])
print('---------------------------------------------------------------------------------')
print(list_all_normalized[0])
    

Para verificar que se cambiaron bien los registros:
{'País': 'Bolivia', 'Tipo de contrato': 'Full-Time', 'Salario mensual o retiro BRUTO (en tu moneda local)': '13500', 'Salario mensual o retiro NETO (en tu moneda local)': '12000', 'Pagos en dólares': '', '¿Cuál fue el último valor de dólar que tomaron?': '', '¿Qué tan conforme estás con tu sueldo?': '3', 'Cómo creés que está tu sueldo con respecto al último semestre': '3', 'Recibís algún tipo de bono': 'Menos de un sueldo', 'A qué está atado el bono': 'Performance de la compañía', '¿Tuviste ajustes por inflación durante 2021?': 'No', '¿De qué % fue el ajuste total?': '0', '¿En qué mes fue el último ajuste?': '0', 'Trabajo de': 'SysAdmin / DevOps / SRE', 'Años de experiencia': '6', 'Años en la empresa actual': '14', 'Años en el puesto actual': '6', '¿Gente a cargo?': '2', 'Plataformas': 'Amazon Web Services, Azure, Docker, Linux, OpenStack', 'Lenguajes de programación o tecnologías.': 'Bash/Shell', 'Frameworks, herramientas y librerías

2. Printear porcentaje de participación por región, ordenado de mayor a menor. Ej: 

```
- Ciudad Autónoma de Buenos Aires - 59.4%
- Formosa - 0.04%
```

In [300]:
sorted_tuples_Participacion_Region = participation_per_key_DESC(list_all_normalized, 'Ciudad')
register_total = participation_count(sorted_tuples_Participacion_Region)

for tuple_record in sorted_tuples_Participacion_Region:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')


 - Ciudad autonoma de buenos aires - 51.71%
 - Provincia de buenos aires - 10.1%
 - Gba - 9.2%
 - Cordoba - 7.8%
 - Santa fe - 5.82%
 - Mendoza - 2.18%
 - Montevideo - 1.7%
 - Entre rios - 0.96%
 - Rio negro - 0.76%
 - Tucuman - 0.76%
 - Neuquen - 0.68%
 - Misiones - 0.61%
 - Santiago - 0.58%
 - Chaco - 0.57%
 - Mexico df - 0.47%
 - Lima - 0.45%
 - Salta - 0.42%
 - San juan - 0.35%
 - Corrientes - 0.35%
 - Bogota - 0.28%
 - Chubut - 0.28%
 - Jujuy - 0.28%
 - Medellin - 0.25%
 - Quito - 0.23%
 - La rioja - 0.23%
 - San luis - 0.17%
 - Tierra del fuego - 0.17%
 - Santiago de cali - 0.13%
 - Ciudad guatemala - 0.12%
 - La pampa - 0.12%
 - Santiago del estero - 0.12%
 - Guadalajara - 0.1%
 - Guayaquil - 0.09%
 - Asuncion - 0.09%
 - Santa cruz de la sierra - 0.07%
 - Santa cruz - 0.07%
 - Caracas - 0.07%
 - Queretaro - 0.06%
 - Cochabamba - 0.04%
 - Valparaiso - 0.04%
 - Tunja - 0.04%
 - Manizales - 0.04%
 - San jose - 0.04%
 - Loja - 0.04%
 - Monterrey - 0.04%
 - Veracruz - 0.04%
 - Puebla

In [302]:
#print(sorted_tuples_Participacion_Region)

3. Printear porcentaje de participación por tipo de rol, ordenado de mayor a menor. Ej:  
`- Developer - 39.11 %`

In [303]:
sorted_tuples_Participacion_Rol = participation_per_key_DESC(list_all_normalized, 'Trabajo de')
register_total = participation_count(sorted_tuples_Participacion_Rol)

for tuple_record in sorted_tuples_Participacion_Rol:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')


 - Developer - 40.69%
 - Sysadmin / devops / sre - 11.09%
 - Technical leader - 6.16%
 - Qa / tester - 4.29%
 - Manager / director - 4.28%
 - Project manager - 3.09%
 - Helpdesk - 2.62%
 - Bi analyst / data analyst - 2.45%
 - Architect - 2.23%
 - Consultant - 1.92%
 - Data engineer - 1.76%
 - Business analyst - 1.69%
 - Networking - 1.54%
 - Ux - 1.5%
 - Data scientist - 1.48%
 - Infosec - 1.43%
 - Recruiter / hr - 1.24%
 - Product manager - 1.21%
 - Dba - 0.71%
 - Scrum master - 0.66%
 - Designer - 0.64%
 - Vp / c-level - 0.55%
 - Sales / pre-sales - 0.42%
 - Analista funcional - 0.19%
 - Middleware - 0.17%
 - Finance - 0.17%
 - Technical writer - 0.16%
 - Storage / backup - 0.12%
 - Product owner - 0.09%
 - Docente - 0.07%
 - Team leader - 0.07%
 - Soporte tecnico - 0.07%
 - Software engineer - 0.07%
 - Agile coach - 0.06%
 - Cybersecurity - 0.06%
 - Functional analyst - 0.06%
 - Devops - 0.06%
 - Analista - 0.04%
 - Soporte - 0.04%
 - Qa automation - 0.04%
 - Ui - 0.04%
 - Coordinad

4. Printear porcentaje de respuesta para las preguntas: 
* ¿Contribuís a proyectos Open Source?
* ¿Programás por hobbie?

In [304]:
sorted_tuples_Participacion_Open_Source = participation_per_key_DESC(list_all_normalized, '¿Contribuís a proyectos open source?')
register_total = participation_count(sorted_tuples_Participacion_Open_Source)

print('¿Contribuís a proyectos Open Source?')
for tuple_record in sorted_tuples_Participacion_Open_Source:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

¿Contribuís a proyectos Open Source?
 - No contesta - 49.44%
 - No - 42.99%
 - Si - 7.57%


In [305]:
sorted_tuples_Participacion_Hobbie = participation_per_key_DESC(list_all_normalized, '¿Programás como hobbie?')
register_total = participation_count(sorted_tuples_Participacion_Hobbie)

print('¿Programás por hobbie?')
for tuple_record in sorted_tuples_Participacion_Hobbie:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

¿Programás por hobbie?
 - No contesta - 49.44%
 - Si - 28.34%
 - No - 22.22%


5. Printear porcentaje por seniority según años de experiencia, el mapeo es:
```
Junior: de 0 hasta 2 años.
Semi-Senior: de 2 años inclusive hasta 5 años.
Senior: desde 5 años inclusive.
``` 

In [308]:
dict_Seniority_Years = {'Junior': 0, 'Semi-Senior': 0, 'Senior': 0}

for dict_line in list_all_normalized:
    years_of_experiencie = dict_line['Años de experiencia']
    if years_of_experiencie < 2:
        dict_Seniority_Years['Junior'] += 1
    elif years_of_experiencie >= 5:
        dict_Seniority_Years['Senior'] += 1
    else:
        dict_Seniority_Years['Semi-Senior'] += 1
    
#print(dict_Seniority_Years)

register_total = dict_Seniority_Years['Junior'] + dict_Seniority_Years['Senior'] + dict_Seniority_Years['Semi-Senior']

print('Porcentaje de participación por Seniority:')
for key in dict_Seniority_Years.keys():
    print(f' {key} - {percentage(register_total,dict_Seniority_Years[key])}%')



Porcentaje de participación por Seniority:
 Junior - 15.93%
 Semi-Senior - 26.63%
 Senior - 57.45%


6. Printear porcentaje de personas encuestadas por años en la compañía actual y por años en el puesto actual.

In [319]:
sorted_tuples_Participacion_Anos_Empresa_Actual = participation_per_key_DESC(list_all_normalized, 'Años en la empresa actual')
register_total = participation_count(sorted_tuples_Participacion_Anos_Empresa_Actual)

print('Porcentaje de personas encuestadas por años en la compañía actual:')
for tuple_record in sorted_tuples_Participacion_Anos_Empresa_Actual:
    print(f' - {normalize_INT(tuple_record[0])} años - {percentage(register_total,tuple_record[1])}%')
    
print(f' *Se detectaron valores fuera del rango válido. Se decidió dejarlos expuestos.')

Porcentaje de personas encuestadas por años en la compañía actual:
 - 0 años - 36.76%
 - 1 años - 16.17%
 - 2 años - 14.62%
 - 3 años - 8.39%
 - 4 años - 4.32%
 - 5 años - 4.11%
 - 6 años - 2.61%
 - 10 años - 2.1%
 - 8 años - 1.98%
 - 7 años - 1.66%
 - 9 años - 1.14%
 - 15 años - 0.92%
 - 11 años - 0.87%
 - 12 años - 0.84%
 - 14 años - 0.61%
 - 13 años - 0.61%
 - 16 años - 0.51%
 - 20 años - 0.48%
 - 18 años - 0.25%
 - 17 años - 0.22%
 - 25 años - 0.17%
 - 21 años - 0.17%
 - 22 años - 0.16%
 - 19 años - 0.06%
 - 23 años - 0.06%
 - 28 años - 0.04%
 - 24 años - 0.04%
 - 26 años - 0.04%
 - 30 años - 0.01%
 - 81 años - 0.01%
 - 2021 años - 0.01%
 - 100 años - 0.01%
 - 32 años - 0.01%
 - 35 años - 0.01%
 *Se detectaron valores fuera del rango válido. Se decidió dejarlos expuestos.


In [320]:
sorted_tuples_Participacion_Anos_Puesto_Actual = participation_per_key_DESC(list_all_normalized, 'Años en el puesto actual')
register_total = participation_count(sorted_tuples_Participacion_Anos_Puesto_Actual)

print('Porcentaje de personas encuestadas por años en su puesto actual:')
for tuple_record in sorted_tuples_Participacion_Anos_Puesto_Actual:
    print(f' - {normalize_INT(tuple_record[0])} años - {percentage(register_total,tuple_record[1])}%')

print(f' *Se detectaron valores fuera del rango válido. Se decidió dejarlos expuestos.')

Porcentaje de personas encuestadas por años en su puesto actual:
 - 0 años - 30.44%
 - 1 años - 19.86%
 - 2 años - 16.68%
 - 3 años - 10.34%
 - 4 años - 5.31%
 - 5 años - 5.15%
 - 6 años - 2.81%
 - 10 años - 2.55%
 - 7 años - 1.53%
 - 8 años - 1.53%
 - 12 años - 0.7%
 - 15 años - 0.7%
 - 9 años - 0.57%
 - 11 años - 0.41%
 - 14 años - 0.33%
 - 13 años - 0.28%
 - 20 años - 0.22%
 - 16 años - 0.13%
 - 18 años - 0.12%
 - 25 años - 0.07%
 - 17 años - 0.07%
 - 21 años - 0.04%
 - 19 años - 0.04%
 - 22 años - 0.03%
 - 28 años - 0.03%
 - 80 años - 0.01%
 - 24 años - 0.01%
 - 33 años - 0.01%
 - 27 años - 0.01%
 *Se detectaron valores fuera del rango válido. Se decidió dejarlos expuestos.


7. Printear porcentajes de nivel de educación formal y estado, es decir: % educación secundaria, terciaria, universitaria, postgrado, doctorado, postdoctorado, completo, incompleto y en curso para cada uno.

In [321]:
sorted_tuples_Participacion_Nivel_Estudios = participation_per_keys_Order_by_Keys(list_all_normalized, 'Nivel de estudios alcanzado', 'Estado')
#print(sorted_tuples_Participacion_Nivel_Estudios)
register_total_tuples_Participacion_Nivel_Estudios = participation_count(sorted_tuples_Participacion_Nivel_Estudios)
   
print('Porcentaje de nivel de educación formal y su estado:')
for tuple_record in sorted_tuples_Participacion_Nivel_Estudios:
    print(f' - {tuple_record[0][0]} {tuple_record[0][1]} - {percentage(register_total_tuples_Participacion_Nivel_Estudios,tuple_record[1])}%')
    

Porcentaje de nivel de educación formal y su estado:
 - Doctorado Completado - 0.37%
 - Doctorado En curso - 0.29%
 - Doctorado Incompleto - 0.06%
 - Posdoctorado Completado - 0.06%
 - Posdoctorado En curso - 0.03%
 - Posdoctorado Incompleto - 0.03%
 - Posgrado Completado - 4.55%
 - Posgrado En curso - 1.64%
 - Posgrado Incompleto - 0.75%
 - Primario Incompleto - 0.03%
 - Secundario Completado - 5.79%
 - Secundario En curso - 0.14%
 - Secundario Incompleto - 0.46%
 - Terciario Completado - 8.84%
 - Terciario En curso - 4.75%
 - Terciario Incompleto - 3.71%
 - Universitario Completado - 25.19%
 - Universitario En curso - 25.45%
 - Universitario Incompleto - 17.85%


8. Formatear las carreras universitarias:
- Nombres capitalizados
- Reemplazar vocales con tilde por vocales sin tilde.
- Reemplazar `ñ` por `n`
- *lic*, *lic.* por Licenciatura
- *tec*, *tec.* por Tecnicatura
* *cs, *cs.* por Ciencias
* *ed, ed.* por Educación
* Transformaciones que se consideren necesarias

Printear porcentaje según carrera

In [327]:
sorted_tuples_Participacion_Carrera = participation_per_key_DESC(list_all_normalized, 'Carrera')
register_total = participation_count(sorted_tuples_Participacion_Carrera)

print('Porcentaje de participación por Carrera:')
for tuple_record in sorted_tuples_Participacion_Carrera:
    print(f' - {(tuple_record[0])} - {percentage(register_total,tuple_record[1])}%')

Porcentaje de participación por Carrera:
 - Ingenieria en sistemas de informacion - 19.0%
 - Ingenieria en informatica - 11.34%
 - Analista de sistemas - 8.68%
 - Licenciatura en ciencias de la computacion - 5.66%
 - Licenciatura en sistemas de informacion - 5.66%
 - Licenciatura en informatica - 4.3%
 - Tecnicatura en programacion - 3.39%
 - Ingenieria electronica - 2.99%
 - Ingenieria en computacion - 2.81%
 - Diseno grafico - 2.48%
 - Tecnicatura superior en programacion - 2.36%
 - Licenciatura en administracion - 1.72%
 - Tecnicatura en informatica - 1.57%
 - Ingenieria industrial - 1.18%
 - Licenciatura en analisis de sistemas - 1.18%
 - Analista programador - 1.12%
 - Ingenieria en software - 1.06%
 - Analista programador universitario - 1.0%
 - Ingenieria en telecomunicaciones - 0.97%
 - Analista de computacion - 0.88%
 - Tecnicatura superior en sistemas informaticos - 0.88%
 - Diseno multimedial - 0.85%
 - Tecnicatura en sistemas - 0.82%
 - Tecnicatura superior en analisis de s

9. Printear porcentaje de respuesta a las preguntas:
- ¿Realizaste cursos de especialización?
- Si realizaste, ¿Quién pagó por los cursos de especialización?

In [328]:
dict_Curso_Espec = {'Si': 0, 'No': 0}

for dict_line in list_all_normalized:
    courses = dict_line['Realizaste cursos de especialización']
    if 'Si' in courses:
        dict_Curso_Espec['Si'] += 1
    else:
        dict_Curso_Espec['No'] += 1

register_total = dict_Curso_Espec['Si'] + dict_Curso_Espec['No']

print('Realizaron cursos de especialización:')  
for key in dict_Curso_Espec.keys():
    print(f' {key} - {percentage(register_total,dict_Curso_Espec[key])}%')
    
    


Realizaron cursos de especialización:
 Si - 39.85%
 No - 60.15%


In [329]:
dict_Pago_Curso = {'Empleador': 0, 'Particular': 0, 'A medias': 0}

for dict_line in list_all_normalized:
    courses = dict_line['Realizaste cursos de especialización']
    if 'empleador' in courses and 'particular' in courses:
        dict_Pago_Curso['A medias'] += 1
    elif 'particular' in courses: 
        dict_Pago_Curso['Particular'] += 1
    else: 
        dict_Pago_Curso['Empleador'] += 1

register_total = dict_Pago_Curso['Empleador'] + dict_Pago_Curso['Particular'] + dict_Pago_Curso['A medias']

print('Si realizó, ¿Quién pagó por los cursos de especialización?')  
for key in dict_Pago_Curso.keys():
    print(f' {key} - {percentage(register_total,dict_Pago_Curso[key])}%')

Si realizó, ¿Quién pagó por los cursos de especialización?
 Empleador - 64.32%
 Particular - 25.33%
 A medias - 10.35%


10. Printear porcentaje por identidad de género y personas con discapacidad

In [330]:
dict_Genero = {'Hombre': 0, 'Mujer': 0, 'No binario': 0, 'Transgénero': 0, 'Prefiere no decir': 0}
# se optó por dejar afuera las respuestas fuera de lugar

for dict_line in list_all_normalized:
    gender = dict_line['Me identifico']
    #print(gender)
    #print(dict_line)[0:100]
    if 'Transgénero' in gender:
        dict_Genero['Transgénero'] += 1
    if 'Hombre' in gender:
        dict_Genero['Hombre'] += 1
    if 'Mujer' in gender: 
        dict_Genero['Mujer'] += 1
    if 'No binario' in gender: 
        dict_Genero['No binario'] += 1
    if 'Prefiere no decir' in gender: 
        dict_Genero['Prefiere no decir'] += 1
   

register_total = dict_Genero['Hombre'] + dict_Genero['Mujer'] + dict_Genero['No binario'] + dict_Genero['Transgénero']

print('Porcentaje de participación por Identidad de Género:')  
for key in dict_Genero.keys():
    print(f' {key} - {percentage(register_total,dict_Genero[key])}%')
    

    

Porcentaje de participación por Identidad de Género:
 Hombre - 81.49%
 Mujer - 17.31%
 No binario - 0.78%
 Transgénero - 0.42%
 Prefiere no decir - 4.34%


In [331]:
sorted_tuples_Participacion_Discapacidad = participation_per_key_DESC(list_all_normalized, '¿Tenés algún tipo de discapacidad?')
register_total = participation_count(sorted_tuples_Participacion_Discapacidad)

print('Porcentaje de participación por Discapacidad:') 
for tuple_record in sorted_tuples_Participacion_Discapacidad:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

Porcentaje de participación por Discapacidad:
 - No - 96.75%
 - Visual - 2.39%
 - Mental - 0.28%
 - Auditiva - 0.2%
 - Motriz - 0.09%
 - Autismo - 0.06%
 - Obesidad - 0.06%
 - Adhd - 0.03%
 - Epilepsia - 0.03%
 - Tdah - 0.03%
 - Asperger - 0.01%
 - Emocional - 0.01%
 - Fonetica - 0.01%
 - El trastorno de ansiedad social (tas) - 0.01%
 - Sensorial/sensitiva - 0.01%
 - Apatia selectiva - 0.01%


11. Salarios: calcular la mediana salarial para cada una de las siguientes categorías:
- Salario según región
- Salario por rol
- Salario por experiencia
- Salario por nivel de formación
- Salario por carrera
- Salario por tecnología
- Salario por lenguaje de programación
- Salario por género

In [332]:
salario_region = dict_values_por_column(list_all_normalized, 'Ciudad', 'Salario mensual o retiro BRUTO (en tu moneda local)')
#print(salario_region)

print('Mediana salarial por Región:')  
for key in salario_region.keys():
    print(f' {key} - U$S {get_mediana(salario_region[key])}')



Mediana salarial por Región:
 Cochabamba - U$S 1163.9
 La paz - U$S 2176.13
 Santa cruz de la sierra - U$S 1339.16
 Santiago - U$S 2452.53
 Maipo - U$S 1576.62
 Biobio - U$S 1446.85
 Valparaiso - U$S 1868.59
 Cachapoal - U$S 3970.76
 Talca - U$S 5372.2
 Valdivia - U$S 2440.85
 Bogota - U$S 1007.31
 Medellin - U$S 1262.92
 Tunja - U$S 881.4
 Santiago de cali - U$S 629.57
 Cucuta - U$S 345.0
 Ibague - U$S 453.29
 Cartagena de indias - U$S 428.11
 Pereira - U$S 252.21
 Bucaramanga - U$S 1259.14
 Manizales - U$S 856.21
 Neiva - U$S 0.15
 Monteria - U$S 1133.22
 Arauca - U$S 377.74
 Santa cruz - U$S 868.21
 San jose - U$S 756.22
 Purral - U$S 1052.66
 Heredia - U$S 1701.5
 Provincia de ciudad de la habana - U$S 244.37
 Sancti spiritus - U$S 242.29
 Guayaquil - U$S 0.04
 Quito - U$S 0.06
 Salinas - U$S 0.12
 Cuenca - U$S 0.11
 Loja - U$S 0.04
 Ambato - U$S 0.05
 Riobamba - U$S 0.01
 Santa tecla - U$S 616.02
 San salvador - U$S 0.17
 Antigua guatemala - U$S 3529.41
 Ciudad guatemala - U$S 239

In [333]:
salario_rol = dict_values_por_column(list_all_normalized, 'Trabajo de', 'Salario mensual o retiro BRUTO (en tu moneda local)')
#print(salario_rol)

print('Mediana salarial por Rol:')  
for key in salario_rol.keys():
    print(f' {key} - U$S {get_mediana(salario_rol[key])}')

Mediana salarial por Rol:
 Sysadmin / devops / sre - U$S 1389.13
 Developer - U$S 1302.31
 Architect - U$S 1996.87
 Networking - U$S 1051.26
 Business analyst - U$S 1241.54
 Technical leader - U$S 2048.97
 Vp / c-level - U$S 3399.03
 Consultant - U$S 1128.67
 Qa / tester - U$S 1111.3
 Scrum master - U$S 1735.54
 Alliances manager (indirect sales) - U$S 3176.61
 Data engineer - U$S 1614.86
 Project manager - U$S 1831.91
 Bi analyst / data analyst - U$S 1116.69
 Data quality - U$S 2919.67
 Analista - U$S 1093.94
 Infosec - U$S 1438.52
 Ux - U$S 1276.26
 Aprentice - U$S 0.23
 Helpdesk - U$S 651.15
 Docente - U$S 554.02
 Product manager - U$S 2131.45
 Ingeniero infraestructuras de virtualizacion - U$S 931.76
 Manager / director - U$S 2778.26
 Data scientist - U$S 1649.59
 Teacher - U$S 755.48
 Profesor de teatro - U$S 503.66
 Ti - U$S 1052.66
 Mix entre sysadmin y developer pasando por dba - U$S 0.05
 Dba - U$S 1562.77
 Chef - U$S 0.01
 Designer - U$S 682.0
 Artesanal - U$S 0.02
 Technical

In [339]:
salario_experiencia = dict_values_por_column(list_all_normalized, 'Años de experiencia', 'Salario mensual o retiro BRUTO (en tu moneda local)')
#print(salario_experiencia)

print('Mediana salarial por Experiencia:')  
for key in salario_experiencia.keys():
    print(f' {key} - U$S {get_mediana(salario_experiencia[key])}')

Mediana salarial por Experiencia:
 6.0 - U$S 1562.77
 12.0 - U$S 1823.23
 8.0 - U$S 1623.55
 10.0 - U$S 1736.41
 0.0 - U$S 642.47
 16.0 - U$S 1736.41
 9.0 - U$S 1797.19
 7.0 - U$S 1653.93
 15.0 - U$S 1866.64
 4.0 - U$S 1302.31
 3.0 - U$S 1284.95
 5.0 - U$S 1502.0
 20.0 - U$S 1897.03
 2.0 - U$S 1041.85
 27.0 - U$S 2972.16
 1.0 - U$S 842.16
 22.0 - U$S 1996.87
 14.0 - U$S 2170.52
 30.0 - U$S 1949.83
 11.0 - U$S 1736.41
 24.0 - U$S 2040.28
 13.0 - U$S 1910.05
 3.5 - U$S 1503.08
 17.0 - U$S 1897.03
 18.0 - U$S 1756.64
 29.0 - U$S 1799.64
 23.0 - U$S 2700.12
 21.0 - U$S 1949.12
 19.0 - U$S 1857.96
 82.0 - U$S 27953495405.11
 2.5 - U$S 1189.44
 1.8 - U$S 1601.93
 25.0 - U$S 1901.37
 1.5 - U$S 1041.85
 33.0 - U$S 1519.36
 7.5 - U$S 1525.37
 35.0 - U$S 1454.24
 4.5 - U$S 2257.34
 34.0 - U$S 1354.4
 40.0 - U$S 0.96
 1.6 - U$S 560.07
 31.0 - U$S 2756.56
 32.0 - U$S 1890.14
 4.75 - U$S 1389.13
 1.9 - U$S 1114.71
 28.0 - U$S 1111.3
 26.0 - U$S 2517.8
 38.0 - U$S 2170.52
 9.5 - U$S 1377.65
 36.0 - 

In [340]:
salario_formacion = dict_values_por_column(list_all_normalized, 'Estado', 'Salario mensual o retiro BRUTO (en tu moneda local)')
#print(salario_formacion)

print('Mediana salarial por Nivel de Formación:')  
for key in salario_formacion.keys():
    print(f' {key} - U$S {get_mediana(salario_formacion[key])}')

Mediana salarial por Nivel de Formación:
 Completado - U$S 1495.92
 Incompleto - U$S 1562.77
 En curso - U$S 1137.35


In [341]:
salario_carrera = dict_values_por_column(list_all_normalized, 'Carrera', 'Salario mensual o retiro BRUTO (en tu moneda local)')
#print(salario_carrera)

print('Mediana salarial por Carrera:')  
for key in salario_carrera.keys():
    print(f' {key} - U$S {get_mediana(salario_carrera[key])}')

Mediana salarial por Carrera:
 Analista de sistemas - U$S 1354.4
 Ingenieria en informatica - U$S 1606.18
 Ingenieria en sistemas de informacion - U$S 1490.87
 Licenciatura en ciencias de la computacion - U$S 1823.23
 Bootcamp en programacion web full stack - U$S 1427.77
 Licenciatura en sistemas de informacion - U$S 1484.63
 Ingenieria ejecucion electronica - U$S 3970.76
 Ingenieria industrial - U$S 1594.03
 Ingenieria en software - U$S 1736.41
 Tecnicatura en informatica - U$S 1128.67
 Ingenieria electronica - U$S 1337.04
 Tecnicatura en sistemas - U$S 842.16
 Ingenieria en telecomunicaciones - U$S 1710.37
 Sistemas de informacion - U$S 1580.14
 Economia - U$S 1649.59
 Ingenieria mecatronica - U$S 2014.62
 Ingenieria administrativa - U$S 1692.28
 Ingenieria en computacion - U$S 1423.86
 Tecnico medio en soporte en informatica - U$S 629.18
 Licenciatura en ciencias fiscales mencion aduana y comercio - U$S 0.02
 Licenciatura en informatica - U$S 1425.88
 Hacking - U$S 2530.41
 Licencia

In [344]:
salario_plataformas = dict_values_por_column(list_all_normalized, 'Plataformas', 'Salario mensual o retiro BRUTO (en tu moneda local)')
#print(salario_plataformas)

print('Mediana salarial por Tecnologías:')  
for key in salario_plataformas.keys():
    print(f' {key} - U$S {get_mediana(salario_plataformas[key])}')

Mediana salarial por Tecnologías:
 Amazon Web Services, Azure, Docker, Linux, OpenStack - U$S 1965.07
 Amazon Web Services, Heroku, Linux - U$S 2745.62
 AIX, Docker, Firebase, Linux, VMWare, Windows Server - U$S 1528.38
 Amazon Web Services, Azure, Docker, Kubernetes, Linux, SAP, Serverless, VMWare, Windows Server - U$S 858.81
 Arduino o similar, Docker, Linux, VMWare, Windows Server, Mikrotik  - U$S 1163.9
 Amazon Web Services, Docker, Firebase, Heroku, Linux - U$S 1383.89
 Amazon Web Services, Arduino o similar, IBM Cloud / Watson - U$S 727.8
 Windows Server - U$S 1041.85
 Ninguna de las anteriores - U$S 1172.08
 Google Cloud Platform/App Engine, SAP - U$S 1015.8
 Docker, Google Cloud Platform/App Engine, Kubernetes, Linux - U$S 1627.88
 Amazon Web Services, Docker, Linux - U$S 1806.15
 Docker, Kubernetes, Linux, OpenShift, SAP, VMWare - U$S 1576.62
 Amazon Web Services, Azure, Docker, Google Cloud Platform/App Engine, Linux - U$S 2538.59
 Docker, Linux - U$S 1172.08
 Azure, Docker, 

In [345]:
salario_tecnologia = dict_values_por_column(list_all_normalized, 'Lenguajes de programación o tecnologías.', 'Salario mensual o retiro BRUTO (en tu moneda local)')
#print(salario_tecnologia)

print('Mediana salarial por Lenguaje de Programación:')  
for key in salario_tecnologia.keys():
    print(f' {key} - U$S {get_mediana(salario_tecnologia[key])}')

Mediana salarial por Lenguaje de Programación:
 Bash/Shell - U$S 1302.31
 Bash/Shell, Ruby - U$S 2831.98
 Bash/Shell, Java, SQL, VBA - U$S 1206.98
 C, Java, Javascript, Python - U$S 858.81
 .NET, Arduino, C#, CSS, HTML, Java, Javascript, Kotlin, PHP, SQL - U$S 1163.9
 Javascript, Python, SQL, TypeScript - U$S 2778.26
 .NET, C++, C#, HTML, Java, Javascript - U$S 727.8
 .NET, Arduino, C, C++, C# - U$S 356.62
 Swift - U$S 1736.41
 .NET, C#, SQL - U$S 1302.31
 Python, SQL - U$S 1406.49
 CSS, Go, HTML, PHP, Python, SQL - U$S 2386.43
 HTML, Javascript, PHP - U$S 1063.55
 CSS, HTML, Javascript, PHP, SQL, TypeScript - U$S 1701.68
 Ninguno de los anteriores - U$S 1128.67
 Java - U$S 1389.13
 Bash/Shell, Go, PHP, Python, SQL - U$S 781.38
 HTML, Javascript, Ruby - U$S 1471.67
 Python - U$S 1509.81
 Bash/Shell, Go, HTML, Java, Javascript, Perl, Python, SQL - U$S 4554.69
 Java, SQL - U$S 1302.31
 Bash/Shell, CSS, HTML, Java, Javascript, SQL - U$S 1475.95
 CSS, Javascript, Ruby, SQL - U$S 1427.77
 J

In [346]:
salario_genero = dict_values_por_column(list_all_normalized, 'Me identifico', 'Salario mensual o retiro BRUTO (en tu moneda local)')
#print(salario_genero.keys())

print('Mediana salarial por Género:')  
for key in salario_genero.keys():
    print(f' {key} - U$S {get_mediana(salario_genero[key])}')

Mediana salarial por Género:
 Hombre - U$S 1432.54
 Mujer - U$S 1093.94
 Prefiere no decir - U$S 1380.45
 Inválido - U$S 1519.36
 No binario - U$S 1215.49
 Transgénero - U$S 1295.97


12. Printear porcentaje y salario según tipos de contrato .

In [347]:
sorted_tuples_Participacion_Contrato = participation_per_key_DESC(list_all_normalized, 'Tipo de contrato')
register_total = participation_count(sorted_tuples_Participacion_Contrato)

salario_contrato = dict_values_por_column(list_all_normalized, 'Tipo de contrato', 'Salario mensual o retiro BRUTO (en tu moneda local)')

print('Porcentaje y mediana de salario según Tipo de Contrato:')
for tuple_record in sorted_tuples_Participacion_Contrato:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}% - U$S {get_mediana(salario_contrato[tuple_record[0]])}')



Porcentaje y mediana de salario según Tipo de Contrato:
 - Full-Time - 82.62% - U$S 1371.77
 - Remoto (empresa de otro país) - 7.12% - U$S 4341.03
 - Part-Time - 4.37% - U$S 564.33
 - Tercerizado (trabajo a través de consultora o agencia) - 3.04% - U$S 1141.69
 - Freelance - 2.59% - U$S 1093.94
 - Participación societaria en una cooperativa - 0.26% - U$S 1111.3


13. En base a los resultados obtenidos confeccionar conclusiones respecto a:

- Rol vs sueldos
- Nivel de educación alcanzada vs sueldos
- Género vs sueldos
- Tecnologías populares

En base a los números analizados se obtuvieron conclusiones para cada caso consultado.

*ROL vs. SALARIOS*

Si analizamos el rol versus el sueldo promedio de cada uno, claramente se ve una disparidad en cuanto aquellos trabajos referidos a la tecnología, a roles en boga, en relación a los más tradicionales, por llamarlos de alguna manera.

Los cargos tradicionales, como atención al cliente, redacción y administración, tienen sueldos muy inferiores en comparación a los nuevos roles, es el caso de un Technical leader, Data Quality, Alliance Manager por nombrar algunos.

Aquellos cargos sumamente técnicos son los que presentan un salario muy por encima de la media. 

*NNIVEL DE EDUCACIÓN ALCANZADA vs. SALARIOS*

En relación al nivel de formación alcanzada no se ve una diferencia entre aquellos que completaron o no su formación. Si, con aquellos que siguen cursando, de todas maneras, no es una gran diferencia.

Pese a esa similitud, vale señalar que es apenas unos dólares más alto la media de salario para aquellos que tienen incompleta su formación. 

Se puede concluir que independientemente del nivel de formación los sueldos son bastante parejos, esto puede deberse a que en la industria que estamos analizando falta mano de obra, entre otras razones.

*GÉNERO vs. SALARIOS*

Aquí se ve claramente una brecha entre los salarios analizados desde el punto de vista del género.
El salario promedio de las mujeres es inferior al resto de los grupos.

Por su parte, los que indicaron ser hombres, así como aquellos que respondieron cosas fuera de lugar (en su gran mayoría haciendo referencia a su masculinidad) son los que tienen los salarios más alto.

Quedando en un escalón intermedio lo que prefirieron no responder, los no binarios y los transgéneros.

*TECNOLOGÍAS MAS POPULARES*

La variedad de plataformas utilizadas es muy amplia. Queda en evidencia que la mayoría de las personas no usa una sola en forma exclusiva, por el contrato conocen y utilizan más de una.

Las tecnologías mas populares dentro del análisis son Amazon, Linux y Windows.

*De la información se pueden deducir infinitas conclusiones más, se quizo mostrar las más notorias.


In [385]:
dict_Plataforma = {'GeneXus': 0, 'Jenkins': 0, '*BSD': 0, '.NET': 0, 'Adobe': 0, 'Amazon Web Services': 0, 'Apple/iOS': 0, 'Google': 0, 'IBM': 0, 'Linux': 0, 'Oracle': 0, 'Power BI': 0, 'Salesforce': 0, 'SAP': 0, 'SQL': 0, 'Windows': 0, 'Huawei': 0, 'Otras': 0}
# se optó por mencionar las más conocidas en la industria, y en otras se agruan el resto.


for dict_line in list_all_normalized:
    platform = dict_line['Plataformas'].lower()
    if 'genexus' in platform:
        dict_Plataforma['GeneXus'] += 1
    if 'jenkins' in platform:
        dict_Plataforma['Jenkins'] += 1
    if 'bsd' in platform or '*bsd' in platform: 
        dict_Plataforma['*BSD'] += 1
    if 'net' in platform or '.net' in platform: 
        dict_Plataforma['.NET'] += 1
    if 'adobe' in platform: 
        dict_Plataforma['Adobe'] += 1
    if 'amazon web services' in platform or 'Amazon' in platform or 'aws' in platform:
        dict_Plataforma['Amazon Web Services'] += 1
    if 'apple/ios' in platform or 'apple' in platform or 'ios' in platform or 'apple ios' in platform:
        dict_Plataforma['Apple/iOS'] += 1
    if 'google' in platform: 
        dict_Plataforma['Google'] += 1
    if 'ibm' in platform or '400' in platform or 'ibm/400' in platform or 'ibm ' in platform or 'ibm 400' in platform: 
        dict_Plataforma['IBM'] += 1
    if 'linux' in platform: 
        dict_Plataforma['Linux'] += 1
    if 'oracle' in platform:
        dict_Plataforma['Oracle'] += 1
    if 'power bi' in platform:
        dict_Plataforma['Power BI'] += 1
    if 'salesforce' in platform or 'sales force' in platform: 
        dict_Plataforma['Salesforce'] += 1
    if 'sap' in platform or 'sap ' in platform: 
        dict_Plataforma['SAP'] += 1
    if 'sql' in platform: 
        dict_Plataforma['SQL'] += 1
    if 'windows' in platform: 
        dict_Plataforma['Windows'] += 1
    if 'huawei' in platform: 
        dict_Plataforma['Huawei'] += 1
    else:
        dict_Plataforma['Otras'] += 1

register_total = dict_Plataforma['GeneXus'] + dict_Plataforma['Jenkins'] + dict_Plataforma['*BSD'] + dict_Plataforma['.NET'] + dict_Plataforma['Adobe'] + dict_Plataforma['Amazon Web Services'] + dict_Plataforma['Apple/iOS'] + dict_Plataforma['Google'] + dict_Plataforma['IBM'] + dict_Plataforma['Linux'] + dict_Plataforma['Oracle'] + dict_Plataforma['Power BI'] + dict_Plataforma['Salesforce'] + dict_Plataforma['SAP'] + dict_Plataforma['SQL'] + dict_Plataforma['Windows'] + dict_Plataforma['Huawei'] + dict_Plataforma['Otras']
                                                                                                                                      
print('Participación de participación por Plataformas:')  
for key in dict_Plataforma.keys():
    print(f' {key} - {percentage(register_total,dict_Plataforma[key])}%')

Participación de participación por Plataformas:
 GeneXus - 0.03%
 Jenkins - 0.05%
 *BSD - 0.39%
 .NET - 6.66%
 Adobe - 0.01%
 Amazon Web Services - 12.62%
 Apple/iOS - 0.08%
 Google - 4.67%
 IBM - 0.81%
 Linux - 18.24%
 Oracle - 0.21%
 Power BI - 0.03%
 Salesforce - 0.25%
 SAP - 2.01%
 SQL - 0.05%
 Windows - 9.59%
 Huawei - 0.03%
 Otras - 44.27%


14. Generar gráficos de barra para mostrar los resultados mencionados anteriormente usando strings, ej:

```
developer | -----------------------------------------------
sysadmin  | ---------------------------
QA        | ------------------
          | ..................5%.......10%...............40%
```

In [348]:
register_total = dict_Seniority_Years['Junior'] + dict_Seniority_Years['Senior'] + dict_Seniority_Years['Semi-Senior']

print('Porcentaje de participación por seniority:')
for key in dict_Seniority_Years.keys():
    print(f' {add_spaces(key,12)} | {number_to_barchart(percentage(register_total,dict_Seniority_Years[key]))} {percentage(register_total,dict_Seniority_Years[key])}% ')

Porcentaje de participación por seniority:
 Junior       | --------------- 15.93% 
 Semi-Senior  | -------------------------- 26.63% 
 Senior       | --------------------------------------------------------- 57.45% 


In [349]:
register_total = participation_count(sorted_tuples_Participacion_Open_Source)


print('¿Contribuís a proyectos Open Source?')
for tuple_record in sorted_tuples_Participacion_Open_Source:
    print(f' {add_spaces(tuple_record[0],12)} | {number_to_barchart(percentage(register_total,tuple_record[1]))} {percentage(register_total,tuple_record[1])}%')

¿Contribuís a proyectos Open Source?
 No contesta  | ------------------------------------------------- 49.44%
 No           | ------------------------------------------ 42.99%
 Si           | ------- 7.57%


In [358]:
register_total = dict_Genero['Hombre'] + dict_Genero['Mujer'] + dict_Genero['No binario'] + dict_Genero['Transgénero']

print('Porcentaje de participación por Identidad de Género:')  
for key in dict_Genero.keys():
    print(f' {add_spaces(key,18)} | {number_to_barchart(percentage(register_total,dict_Genero[key]))} {percentage(register_total,dict_Genero[key])}%')

Porcentaje de participación por Identidad de Género:
 Hombre             | --------------------------------------------------------------------------------- 81.49%
 Mujer              | ----------------- 17.31%
 No binario         |  0.78%
 Transgénero        |  0.42%
 Prefiere no decir  | ---- 4.34%
